<a href="https://colab.research.google.com/github/ikedachin/study_huggingface/blob/main/phi_15_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HuggingfaceからMicrosoftのPhi-1.5を使ってみよう

https://huggingface.co/microsoft/phi-1_5

In [ ]:
!pip install -q huggingface_hub
!pip install -q transformers[torch]==4.42.3
!pip install -q datasets==2.20.0
!pip install -q accelerate==0.32.1
!pip install -q ipywidgets

# #01 ライブラリのインポート

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


# #02 GPU / MPS / CPUの選択

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
device

# #03 トークナイザーとモデルのダウンロード

In [ ]:
torch.set_default_device(device)

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5")

In [ ]:
# トークナイザの語彙数を確認する
tokenizer.vocab_size

In [ ]:
# モデルの構造を見てみる
model.base_model


# #04 出力を確認しながら推論させてみよう
## #04-1 トークナイザーを使って、文章をトークンIDに変換する

In [ ]:
input_text = 'please tell me how to play a fiddle.'

In [ ]:
inputs = tokenizer(input_text, return_tensors="pt", return_attention_mask=False)
inputs

In [ ]:
inputs.input_ids[0]

## #04-2 トークンIDを文章に戻す方法を確認

In [ ]:
tokenizer.decode(inputs.input_ids[0])

## #04-3 Phi-1.5モデルに入力トークンIDを渡して出力させよう

In [ ]:
outputs = model.generate(inputs.input_ids, max_length=200)
# outputs = model.generate(**inputs, max_length=200) # <- こちらの書き方でもOK
outputs

In [ ]:
outputs.size()

## #04-4 モデルから出てきたトークンIDを文章に戻す

In [ ]:
output_text = tokenizer.batch_decode(outputs)[0]
print(output_text)


# #05 DXクエストについて聞いてみよう

In [ ]:
input_text = 'Please tell me about manabi-DX-Quest.'

In [ ]:
inputs = tokenizer(input_text, return_tensors="pt", return_attention_mask=False)
outputs = model.generate(inputs.input_ids, max_length=200)
output_text = tokenizer.batch_decode(outputs)[0]
print(output_text)

# #06 プログラムを書かせてみよう

## #06-1 Pythonの関数の書き方のおさらい

In [ ]:
# まず、関数の例
def add_num(a, b):
    '''引数の二つの値を足す。   <- Docstringsという
    '''
    c = a + b
    return c


In [ ]:
added_number = add_num(4, 5)
added_number

In [ ]:
# 1からnまでの数字で素数をプリントする関数

text = '''
def print_prime(n):
    """Print all primes between 1 and n
    """
'''

In [ ]:
# # a/bをした時に整数ならinteger、小数点があるならfloatと返す関数
# text = '''
# def check_division_type(a, b):
#     """When performing a/b, return "integer" if the result is an integer, and "float" if the result has a decimal point.
#     """
# '''

In [ ]:
inputs = tokenizer(text, return_tensors='pt', return_attention_mask=False)
outputs = model.generate(inputs.input_ids, max_length=400)
text = tokenizer.batch_decode(outputs)[0]
print(text)

In [ ]:
# 出力をコピーして実行してみる

# #07 ファインチューニングに向けて
## #07-1 モデルの設定を確認する

In [ ]:
from transformers import AutoConfig

phi15_config = AutoConfig.from_pretrained("microsoft/phi-1_5")
phi15_config

## #07-2 datasetを使ってみる

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset('ag_news')

In [ ]:
dataset

In [ ]:
dataset['train']['text']

In [ ]:
dataset['train']['label']

### #07-2-1 最初の100データだけを抜き取る

In [ ]:
dataset['train'] = dataset['train'].select(range(100))
len(dataset['train']['text'])

### #07-2-2 便利なmap関数

In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
# 関数を容易する
def tokenizer_func(example):
    return tokenizer(example['text'], padding=True, truncation=True)

In [ ]:
pretreated_dataset = dataset.map(tokenizer_func, batched=True)
pretreated_dataset

In [ ]:
# 最初の一つだけ見てみる
pretreated_dataset_0 =pretreated_dataset['train'][0]
pretreated_dataset_0

In [ ]:
pretreated_dataset_0['text']

In [ ]:
pretreated_dataset_0['input_ids']

In [ ]:
pretreated_dataset_0['attention_mask']

## #07-3 データセットのストリーミング

In [ ]:
import datasets
from datasets import load_dataset

In [ ]:
# その1
dataset = load_dataset('oscar-corpus/OSCAR-2201', 'en', split='train', streaming=True)

In [ ]:
for d in dataset:
    print(d) # prints documents

In [ ]:
# その2
dataset = load_dataset("oscar-corpus/OSCAR-2201",
                        # use_auth_token=True, # required
                        language="ar",
                        streaming=True, # optional
                        split="train", # optional, but the dataset only has a train split
                        trust_remote_code=True)




In [ ]:
for d in dataset:
    print(d) # prints documents

# #08 参考：HuggingFaceへのログイン
モデルやデータセットをアップロードするにはログインが必要

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# コマンドラインでの実行方法
# huggingface-cli login